Practical 6

Aim : Write a program for implementing L1, L2 regularization, early stopping, dropout with value 0.2.
Consider the SONAR dataset. Create a sequential model, with 3 hidden dense layers.
Compare the training and testing accuracies of all models.


In [1]:
# importing libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout
from keras import regularizers, optimizers
from tensorflow.keras.regularizers import l1, l2
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

In [3]:
dataset = pd.read_csv("sonar.csv")
dataset

,attribute_1,attribute_2,attribute_3,attribute_4,attribute_5,attribute_6,attribute_7,attribute_8,attribute_9,attribute_10,...,attribute_52,attribute_53,attribute_54,attribute_55,attribute_56,attribute_57,attribute_58,attribute_59,attribute_60,Class
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,Rock
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,Rock
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,Rock
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,Rock
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,Rock
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0.0187,0.0346,0.0168,0.0177,0.0393,0.1630,0.2028,0.1694,0.2328,0.2684,...,0.0116,0.0098,0.0199,0.0033,0.0101,0.0065,0.0115,0.0193,0.0157,Mine
204,0.0323,0.0101,0.0298,0.0564,0.0760,0.0958,0.0990,0.1018,0.1030,0.2154,...,0.0061,0.0093,0.0135,0.0063,0.0063,0.0034,0.0032,0.0062,0.0067,Mine
205,0.0522,0.0437,0.0180,0.0292,0.0351,0.1171,0.1257,0.1178,0.1258,0.2529,...,0.0160,0.0029,0.0051,0.0062,0.0089,0.0140,0.0138,0.0077,0.0031,Mine
206,0.0303,0.0353,0.0490,0.0608,0.0167,0.1354,0.1465,0.1123,0.1945,0.2354,...,0.0086,0.0046,0.0126,0.0036,0.0035,0.0034,0.0079,0.0036,0.0048,Mine


In [4]:
data = dataset.drop("Class", axis = 'columns')
data = data.to_numpy()
data

array([[0.02  , 0.0371, 0.0428, ..., 0.0084, 0.009 , 0.0032],
       [0.0453, 0.0523, 0.0843, ..., 0.0049, 0.0052, 0.0044],
       [0.0262, 0.0582, 0.1099, ..., 0.0164, 0.0095, 0.0078],
       ...,
       [0.0522, 0.0437, 0.018 , ..., 0.0138, 0.0077, 0.0031],
       [0.0303, 0.0353, 0.049 , ..., 0.0079, 0.0036, 0.0048],
       [0.026 , 0.0363, 0.0136, ..., 0.0036, 0.0061, 0.0115]])

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
target = le.fit_transform(dataset['Class'])
target

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [6]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

In [7]:
len(X_train)

166

In [8]:
input_dim = X_train.shape[1]
input_dim

60

Simple model

In [30]:
# defining model
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(60,) ))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

#compiling the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#fitting the model to training data
model.fit(X_train,y_train,epochs = 50, validation_split=0.1)

Epoch 1/50
5/5 [==============================] - 2s 92ms/step - loss: 0.6988 - accuracy: 0.4564 - val_loss: 0.6879 - val_accuracy: 0.5882
Epoch 2/50
5/5 [==============================] - 0s 15ms/step - loss: 0.6757 - accuracy: 0.5839 - val_loss: 0.6535 - val_accuracy: 0.6471
Epoch 3/50
5/5 [==============================] - 0s 17ms/step - loss: 0.6569 - accuracy: 0.6174 - val_loss: 0.6606 - val_accuracy: 0.5882
Epoch 4/50
5/5 [==============================] - 0s 20ms/step - loss: 0.6395 - accuracy: 0.6779 - val_loss: 0.6615 - val_accuracy: 0.5882
Epoch 5/50
5/5 [==============================] - 0s 18ms/step - loss: 0.6200 - accuracy: 0.7718 - val_loss: 0.6643 - val_accuracy: 0.6471
Epoch 6/50
5/5 [==============================] - 0s 18ms/step - loss: 0.5980 - accuracy: 0.7584 - val_loss: 0.6482 - val_accuracy: 0.6471
Epoch 7/50
5/5 [==============================] - 0s 18ms/step - loss: 0.5742 - accuracy: 0.7517 - val_loss: 0.6601 - val_accuracy: 0.5882
Epoch 8/50
5/5 [===========

In [10]:
#evaluating training accuracy and loss
_, simpleTrainAcc = model.evaluate(X_train,y_train)

6/6 [==============================] - 0s 4ms/step - loss: 0.1675 - accuracy: 0.9639


In [12]:
#evaluating testing accuracy and loss
_, simpleTestAcc = model.evaluate(X_test,y_test)

2/2 [==============================] - 0s 12ms/step - loss: 0.5994 - accuracy: 0.8333


Model with L1 regularization


In [13]:
modelL1 = Sequential()
modelL1.add(Dense(128, activation='relu', input_shape=(60,) ))
modelL1.add(Dense(64, activation='relu'))
modelL1.add(Dense(32, activation='relu', kernel_regularizer= regularizers.l1(0.001)))
modelL1.add(Dense(1, activation='sigmoid'))

#defining optimizer with learning rate
optimizer = optimizers.Adam(learning_rate=0.001)

#compiling the model
modelL1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#fitting the model to training data
modelL1.fit(X_train,y_train,epochs = 50)

Epoch 1/50
6/6 [==============================] - 2s 8ms/step - loss: 0.9499 - accuracy: 0.5120
Epoch 2/50
6/6 [==============================] - 0s 9ms/step - loss: 0.9033 - accuracy: 0.6265
Epoch 3/50
6/6 [==============================] - 0s 8ms/step - loss: 0.8721 - accuracy: 0.6928
Epoch 4/50
6/6 [==============================] - 0s 6ms/step - loss: 0.8471 - accuracy: 0.6807
Epoch 5/50
6/6 [==============================] - 0s 9ms/step - loss: 0.8135 - accuracy: 0.7169
Epoch 6/50
6/6 [==============================] - 0s 9ms/step - loss: 0.7870 - accuracy: 0.7711
Epoch 7/50
6/6 [==============================] - 0s 5ms/step - loss: 0.7556 - accuracy: 0.7651
Epoch 8/50
6/6 [==============================] - 0s 7ms/step - loss: 0.7238 - accuracy: 0.7410
Epoch 9/50
6/6 [==============================] - 0s 7ms/step - loss: 0.6894 - accuracy: 0.7651
Epoch 10/50
6/6 [==============================] - 0s 8ms/step - loss: 0.6636 - accuracy: 0.8072
Epoch 11/50
6/6 [======================

In [31]:
print("Training loss and accuracy for L1 regularization")
_, L1TrainAcc = modelL1.evaluate(X_train, y_train)

Training loss and accuracy for L1 regularization
6/6 [==============================] - 0s 4ms/step - loss: 0.2010 - accuracy: 0.9639


In [32]:
print("Testing loss and accuracy for L1 regularization")
_, L1TestAcc = modelL1.evaluate(X_test, y_test)

Testing loss and accuracy for L1 regularization
2/2 [==============================] - 0s 8ms/step - loss: 0.3867 - accuracy: 0.9048


L2 regularization

In [16]:
modelL2 = Sequential()
modelL2.add(Dense(128, activation='relu', input_shape=(60,) ))
modelL2.add(Dense(64, activation='relu'))
modelL2.add(Dense(32, activation='relu', kernel_regularizer= regularizers.l2(0.001)))
modelL2.add(Dense(1, activation='sigmoid'))

#defining optimizer with learning rate
optimizer = optimizers.Adam(learning_rate=0.001)

#compiling the model
modelL2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#fitting the model to training data
modelL2.fit(X_train,y_train,epochs = 50)

Epoch 1/50
6/6 [==============================] - 2s 6ms/step - loss: 0.7307 - accuracy: 0.5120
Epoch 2/50
6/6 [==============================] - 0s 4ms/step - loss: 0.7069 - accuracy: 0.6386
Epoch 3/50
6/6 [==============================] - 0s 4ms/step - loss: 0.6960 - accuracy: 0.7229
Epoch 4/50
6/6 [==============================] - 0s 5ms/step - loss: 0.6748 - accuracy: 0.7229
Epoch 5/50
6/6 [==============================] - 0s 5ms/step - loss: 0.6518 - accuracy: 0.7410
Epoch 6/50
6/6 [==============================] - 0s 4ms/step - loss: 0.6391 - accuracy: 0.7289
Epoch 7/50
6/6 [==============================] - 0s 5ms/step - loss: 0.6096 - accuracy: 0.7831
Epoch 8/50
6/6 [==============================] - 0s 6ms/step - loss: 0.5995 - accuracy: 0.7289
Epoch 9/50
6/6 [==============================] - 0s 7ms/step - loss: 0.5740 - accuracy: 0.7410
Epoch 10/50
6/6 [==============================] - 0s 5ms/step - loss: 0.5507 - accuracy: 0.7771
Epoch 11/50
6/6 [======================

In [17]:
print("Training loss and accuracy for L2 regularization")
_, L2TrainAcc = modelL2.evaluate(X_train, y_train)

Training loss and accuracy for L2 regularization
6/6 [==============================] - 0s 6ms/step - loss: 0.0907 - accuracy: 0.9940


In [18]:
print("Testing loss and accuracy for L2 regularization")
_, L2TestAcc = modelL2.evaluate(X_test, y_test)

Testing loss and accuracy for L2 regularization
2/2 [==============================] - 0s 21ms/step - loss: 0.3289 - accuracy: 0.9048


Early stopping, patience = 5

In [33]:
modelES = Sequential()
modelES.add(Dense(128, activation='relu', input_shape=(60,) ))
modelES.add(Dense(64, activation='relu'))
modelES.add(Dense(32, activation='relu'))
modelES.add(Dense(1, activation='sigmoid'))

optimizer = optimizers.Adam(learning_rate=0.001)

modelES.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='loss', patience=5)

modelES.fit(X_train, y_train, epochs=50,callbacks=[early_stopping])

Epoch 1/50
6/6 [==============================] - 1s 5ms/step - loss: 0.6944 - accuracy: 0.5060
Epoch 2/50
6/6 [==============================] - 0s 6ms/step - loss: 0.6637 - accuracy: 0.6446
Epoch 3/50
6/6 [==============================] - 0s 5ms/step - loss: 0.6404 - accuracy: 0.7229
Epoch 4/50
6/6 [==============================] - 0s 4ms/step - loss: 0.6194 - accuracy: 0.7711
Epoch 5/50
6/6 [==============================] - 0s 5ms/step - loss: 0.5967 - accuracy: 0.8012
Epoch 6/50
6/6 [==============================] - 0s 5ms/step - loss: 0.5680 - accuracy: 0.7711
Epoch 7/50
6/6 [==============================] - 0s 5ms/step - loss: 0.5406 - accuracy: 0.8012
Epoch 8/50
6/6 [==============================] - 0s 5ms/step - loss: 0.5110 - accuracy: 0.7952
Epoch 9/50
6/6 [==============================] - 0s 6ms/step - loss: 0.4904 - accuracy: 0.8193
Epoch 10/50
6/6 [==============================] - 0s 5ms/step - loss: 0.4762 - accuracy: 0.7711
Epoch 11/50
6/6 [======================

In [34]:
print("Training loss and accuracy for early stopping")
_, ESTrainAcc = modelES.evaluate(X_train, y_train)

Training loss and accuracy for early stopping
6/6 [==============================] - 0s 5ms/step - loss: 0.0860 - accuracy: 0.9819


In [35]:
print("Testing loss and accuracy for early stopping")
_, ESTestAcc = modelES.evaluate(X_test, y_test)

Testing loss and accuracy for early stopping
2/2 [==============================] - 0s 11ms/step - loss: 0.5133 - accuracy: 0.8095


Dropout after input layer

In [37]:
modelDO = Sequential()
modelDO.add(Dense(128, activation='relu', input_shape=(60,) ))
Dropout(0.2)
modelDO.add(Dense(64, activation='relu'))
modelDO.add(Dense(32, activation='relu'))
modelDO.add(Dense(1, activation='sigmoid'))

optimizer = optimizers.Adam(learning_rate=0.001)

modelDO.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

modelDO.fit(X_train, y_train, epochs=50)

Epoch 1/50
6/6 [==============================] - 1s 6ms/step - loss: 0.7000 - accuracy: 0.5000
Epoch 2/50
6/6 [==============================] - 0s 5ms/step - loss: 0.6796 - accuracy: 0.6627
Epoch 3/50
6/6 [==============================] - 0s 6ms/step - loss: 0.6698 - accuracy: 0.6386
Epoch 4/50
6/6 [==============================] - 0s 5ms/step - loss: 0.6616 - accuracy: 0.7651
Epoch 5/50
6/6 [==============================] - 0s 5ms/step - loss: 0.6485 - accuracy: 0.6928
Epoch 6/50
6/6 [==============================] - 0s 5ms/step - loss: 0.6327 - accuracy: 0.7349
Epoch 7/50
6/6 [==============================] - 0s 4ms/step - loss: 0.6277 - accuracy: 0.7169
Epoch 8/50
6/6 [==============================] - 0s 4ms/step - loss: 0.6033 - accuracy: 0.7410
Epoch 9/50
6/6 [==============================] - 0s 5ms/step - loss: 0.5892 - accuracy: 0.7229
Epoch 10/50
6/6 [==============================] - 0s 5ms/step - loss: 0.5576 - accuracy: 0.7771
Epoch 11/50
6/6 [======================

In [38]:
print("Training loss and accuracy for drop out")
_, DOTrainAcc = modelDO.evaluate(X_train, y_train)

Training loss and accuracy for drop out
6/6 [==============================] - 0s 6ms/step - loss: 0.0597 - accuracy: 0.9940


In [39]:
print("Testing loss and accuracy for drop out")
_, DOTestAcc = modelDO.evaluate(X_test, y_test)

Testing loss and accuracy for drop out
2/2 [==============================] - 0s 7ms/step - loss: 0.2556 - accuracy: 0.8571


Comparing the 5 models

In [41]:
# storing the values as a dict

compareModels = {'Model': ['Simple', 'L1 regularised', 'L2 regularised', 'Early stopping (p=5)', 'Dropout (0.2)'],
              'Training Accuracy': [simpleTrainAcc, L1TrainAcc, L2TrainAcc, ESTrainAcc, DOTrainAcc],
              'Testing Accuracy': [simpleTestAcc, L1TestAcc, L2TestAcc, ESTestAcc, DOTestAcc]}

table = pd.DataFrame(compareModels)
table

,Model,Training Accuracy,Testing Accuracy
0,Simple,0.963855,0.833333
1,L1 regularised,0.963855,0.904762
2,L2 regularised,0.993976,0.904762
3,Early stopping (p=5),0.981928,0.809524
4,Dropout (0.2),0.993976,0.857143
